In [2]:
import cv2
import numpy as np

def rotate_image(image, angle):
    # 获取图像尺寸
    (h, w) = image.shape[:2]
    # 计算图像中心
    center = (w // 2, h // 2)
    # 获取旋转矩阵
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    # 进行仿射变换
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def estimate_rotation(image):
    # 灰度化
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # 边缘检测
    edges = cv2.Canny(gray, 50, 150)
    # 霍夫线变换
    lines = cv2.HoughLines(edges, 1, np.pi / 180, 100)
    if lines is not None:
        angles = []
        for line in lines:
            rho, theta = line[0]
            angle = theta * 180 / np.pi
            angles.append(angle)
        # 平均角度
        avg_angle = np.mean(angles)
        return avg_angle
    else:
        return 0

# 加载图像
image = cv2.imread('C:\\Users\\22597\\Documents\\GitHub\\boiling\\modified\\img_to_rotate.png')
# 估计旋转角度
angle = estimate_rotation(image)
# 根据角度旋转图像
corrected_image = rotate_image(image, -angle)

cv2.imshow('Corrected Image', corrected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [10]:


import cv2
import numpy as np

def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h))
    return rotated

def estimate_rotation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        cnt = max(contours, key=cv2.contourArea)
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        angle = rect[2]
        # 如果角度大于45度，我们将其转换为90度减去该角度
        if abs(angle) > 45:
            angle = 90 - abs(angle)
        
        return angle
    else:
        return 0

# 加载图像
image = cv2.imread('C:\\Users\\22597\\Documents\\GitHub\\boiling\\modified\\img_to_rotate.png')
# image = cv2.imread('C:\\Users\\22597\\Documents\\GitHub\\boiling\\modified\\img_to_rotate.png', cv2.IMREAD_UNCHANGED)
# 确保图像为RGB
if image.shape[-1] == 4:  # 如果有Alpha通道
    image = cv2.cvtColor(image, cv2.COLOR_BGRA2BGR)
elif image.shape[-1] == 1:  # 如果是灰度图
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# 估计旋转角度
angle = estimate_rotation(image)
# 根据角度旋转图像
corrected_image = rotate_image(image, -angle)

# 显示校正后的图像
cv2.imshow('Corrected Image', corrected_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

C:\Users\22597\AppData\Local\Temp\ipykernel_20348\2463178154.py:20: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)


In [6]:
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
from keras.optimizers import SGD
import keras.backend as K

import os
import math
import cv2
import numpy as np
import requests


class RotateCaptcha():
    def __init__(self):
        # 加载模型
        model_location = os.path.join('.', 'models', 'resnet50_keras2.hdf5')
        self.model = load_model(model_location, custom_objects={'angle_error': self.angle_error})
        self.model.compile(loss='categorical_crossentropy',
                           optimizer=SGD(lr=0.01, momentum=0.9),
                           metrics=[self.angle_error])
        # 图像长宽尺寸
        self.size = (224, 224)

    def showImg(self, image):
        '''
        展示图片
        '''
        cv2.imshow('image', image)
        cv2.waitKey(0)

    def getImgFromDisk(self, imgPath):
        image = cv2.imread(imgPath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        return image

    def predictAngle(self, image):
        diameter = image.shape[0]  # 直径
        side_length = math.floor((diameter / 2) * 1.414)  # 圆内正方形最大边长
        cropped = math.floor((diameter - side_length) / 2)
        image = image[cropped:cropped + side_length, cropped:cropped + side_length]
        image = cv2.resize(image, self.size)

        image = np.expand_dims(image, axis=0)

        x = preprocess_input(image)
        y_pred = np.argmax(self.model.predict(x), axis=1)

        return y_pred[0]

    def rotate(self, image, angle):

        image_size = (image.shape[1], image.shape[0])
        image_center = tuple(np.array(image_size) / 2)

        # 将 OpenCV 3x2旋转矩阵转换为3x3
        rot_mat = np.vstack(
            [cv2.getRotationMatrix2D(image_center, angle, 1.0), [0, 0, 1]]
        )

        rot_mat_notranslate = np.matrix(rot_mat[0:2, 0:2])

        image_w2 = image_size[0] * 0.5
        image_h2 = image_size[1] * 0.5

        # 获取图像角点的旋转坐标
        rotated_coords = [
            (np.array([-image_w2, image_h2]) * rot_mat_notranslate).A[0],
            (np.array([image_w2, image_h2]) * rot_mat_notranslate).A[0],
            (np.array([-image_w2, -image_h2]) * rot_mat_notranslate).A[0],
            (np.array([image_w2, -image_h2]) * rot_mat_notranslate).A[0]
        ]

        # 查找新图像的大小
        x_coords = [pt[0] for pt in rotated_coords]
        x_pos = [x for x in x_coords if x > 0]
        x_neg = [x for x in x_coords if x < 0]

        y_coords = [pt[1] for pt in rotated_coords]
        y_pos = [y for y in y_coords if y > 0]
        y_neg = [y for y in y_coords if y < 0]

        right_bound = max(x_pos)
        left_bound = min(x_neg)
        top_bound = max(y_pos)
        bot_bound = min(y_neg)

        new_w = int(abs(right_bound - left_bound))
        new_h = int(abs(top_bound - bot_bound))


        trans_mat = np.matrix([
            [1, 0, int(new_w * 0.5 - image_w2)],
            [0, 1, int(new_h * 0.5 - image_h2)],
            [0, 0, 1]
        ])

        affine_mat = (np.matrix(trans_mat) * np.matrix(rot_mat))[0:2, :]

        result = cv2.warpAffine(
            image,
            affine_mat,
            (new_w, new_h),
            flags=cv2.INTER_LINEAR
        )

        return result

    def angle_difference(self, x, y):

        return 180 - abs(abs(x - y) - 180)

    def angle_error(self, y_true, y_pred):

        diff = self.angle_difference(K.argmax(y_true), K.argmax(y_pred))
        return K.mean(K.cast(K.abs(diff), K.floatx()))


if __name__ == '__main__':
    rotateCaptcha = RotateCaptcha()
    rotated_image = rotateCaptcha.getImgFromDisk('222.jpg')

    predicted_angle = rotateCaptcha.predictAngle(rotated_image)  # 预测还原角度
    print("需旋转角度：{}".format(predicted_angle))

    corrected_image = rotateCaptcha.rotate(rotated_image, -predicted_angle)  
    rotateCaptcha.showImg(corrected_image)


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '.\models\resnet50_keras2.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)